# Fetching Data from Oanda API Endpoints

Oanda API endpoints can be found at http://developer.oanda.com/rest-live-v20/account-ep/

In [4]:
import requests
import pandas as pd
import json
from dateutil import parser

In [5]:
account_data = pd.read_csv("~/Documents/temp/oanda/testnet_key.txt", sep=" ", header=None)
API_KEY = account_data[0][2]
ACCOUNT_ID = account_data[0][1]
OANDA_URL = account_data[0][0]

In [6]:
session = requests.Session()

## Fetching Data from the Candles Endpoint

In [7]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [8]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [9]:
def set_candles_url(url, pair):
 return f"{url}/instruments/{pair}/candles"

In [10]:
pair_name = "EUR_USD"
url_candles = set_candles_url(OANDA_URL, pair_name)

In [11]:
response = session.get(url_candles, params=params, data=None, headers=None)

In [12]:
response.status_code

200

In [13]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 2032,
   'time': '2022-07-21T23:00:00.000000000Z',
   'bid': {'o': '1.02232', 'h': '1.02298', 'l': '1.02193', 'c': '1.02194'},
   'mid': {'o': '1.02242', 'h': '1.02306', 'l': '1.02201', 'c': '1.02202'},
   'ask': {'o': '1.02251', 'h': '1.02315', 'l': '1.02209', 'c': '1.02211'}},
  {'complete': True,
   'volume': 3991,
   'time': '2022-07-22T00:00:00.000000000Z',
   'bid': {'o': '1.02195', 'h': '1.02210', 'l': '1.02025', 'c': '1.02038'},
   'mid': {'o': '1.02202', 'h': '1.02218', 'l': '1.02034', 'c': '1.02046'},
   'ask': {'o': '1.02210', 'h': '1.02225', 'l': '1.02041', 'c': '1.02053'}},
  {'complete': True,
   'volume': 4707,
   'time': '2022-07-22T01:00:00.000000000Z',
   'bid': {'o': '1.02037', 'h': '1.02076', 'l': '1.01899', 'c': '1.02016'},
   'mid': {'o': '1.02045', 'h': '1.02084', 'l': '1.01908', 'c': '1.02024'},
   'ask': {'o': '1.02053', 'h': '1.02092', 'l': '1.01917', 'c': '1.02031'}},

## Fetching Data from the Instruments Endpoint

In [14]:
url_inst = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [15]:
response = session.get(url_inst, params=params, data=None, headers=None)

In [16]:
response.status_code

200

In [17]:
data = response.json() # dict_keys(['instruments', 'lastTransactionID'])

In [18]:
instruments_list = data["instruments"]

In [19]:
len(instruments_list)

129

In [20]:
instruments_list

[{'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'minimumTradeSize': '0.1',
  'maximumTrailingStopDistance': '10000.0',
  'minimumTrailingStopDistance': '5.0',
  'maximumPositionSize': '0',
  'maximumOrderUnits': '500',
  'marginRate': '0.1',
  'guaranteedStopLossOrderMode': 'DISABLED',
  'tags': [{'type': 'ASSET_CLASS', 'name': 'INDEX'}],
  'financing': {'longRate': '-0.0258',
   'shortRate': '-0.0242',
   'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
    {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
    {'dayOfWeek': 'FRIDAY', 'daysCharged': 3},
    {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
    {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
 {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'trade

In [21]:
# taking a look at the information that's available for every instrument
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [22]:
# list of keys we're intersted in getting 
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 
         'tradeUnitsPrecision', 'marginRate']

In [23]:
instruments_dict = {}
for i in instruments_list:
    # print(i['name'])
    key = i['name']
    instruments_dict[key] = {
        k: i[k] for k in key_i
    }

In [24]:
instruments_dict

{'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'AU200_AUD': {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'GBP_AUD': {'name': 'GBP_AUD',
  'type': 'CURRENCY',
  'displayName': 'GBP/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'SG30_SGD': {'name': 'SG30_SGD',
  'type': 'CFD',
  'displayName': 'Singapore 30',
  'pipLocation': -1,
  'displayPrecision': 2,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  

In [25]:
# fetching particular instrument
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [26]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

## Fetching a candle pair

In [27]:
def fetch_candles(api_url, pair_name, count="10", granularity="H1"):
    url = f"{api_url}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    status = response.status_code
    
    if status == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return status, data

In [28]:
status, data = fetch_candles(OANDA_URL, "EUR_USD", count=20)

In [29]:
status

200

In [30]:
len(data)

20

In [31]:
data

[{'complete': True,
  'volume': 21530,
  'time': '2022-07-21T13:00:00.000000000Z',
  'bid': {'o': '1.02638', 'h': '1.02743', 'l': '1.01527', 'c': '1.02116'},
  'mid': {'o': '1.02646', 'h': '1.02751', 'l': '1.01536', 'c': '1.02124'},
  'ask': {'o': '1.02655', 'h': '1.02759', 'l': '1.01544', 'c': '1.02133'}},
 {'complete': True,
  'volume': 14475,
  'time': '2022-07-21T14:00:00.000000000Z',
  'bid': {'o': '1.02115', 'h': '1.02134', 'l': '1.01675', 'c': '1.01909'},
  'mid': {'o': '1.02123', 'h': '1.02142', 'l': '1.01684', 'c': '1.01916'},
  'ask': {'o': '1.02131', 'h': '1.02151', 'l': '1.01694', 'c': '1.01924'}},
 {'complete': True,
  'volume': 7808,
  'time': '2022-07-21T15:00:00.000000000Z',
  'bid': {'o': '1.01907', 'h': '1.02116', 'l': '1.01816', 'c': '1.01997'},
  'mid': {'o': '1.01916', 'h': '1.02124', 'l': '1.01824', 'c': '1.02004'},
  'ask': {'o': '1.01925', 'h': '1.02133', 'l': '1.01833', 'c': '1.02011'}},
 {'complete': True,
  'volume': 5522,
  'time': '2022-07-21T16:00:00.00000

In [32]:
# getting the prices from the JSON object
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
# loop through the prices and make for each one a column key 
# that looks like mid_c, mid_o, mid_h, mid_c
for p in prices:
    for o in ohlc:
        print(f"{p}_{o}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [33]:
# collecting certain info from the instruments
# and converting it to a data frame
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []
    
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        # loop through the prices and make for each one a column key 
        # that looks like mid_c, mid_o, mid_h, mid_c
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

In [34]:
df = get_candles_df(data)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    19 non-null     datetime64[ns, tzutc()]
 1   volume  19 non-null     int64                  
 2   mid_o   19 non-null     float64                
 3   mid_h   19 non-null     float64                
 4   mid_l   19 non-null     float64                
 5   mid_c   19 non-null     float64                
 6   bid_o   19 non-null     float64                
 7   bid_h   19 non-null     float64                
 8   bid_l   19 non-null     float64                
 9   bid_c   19 non-null     float64                
 10  ask_o   19 non-null     float64                
 11  ask_h   19 non-null     float64                
 12  ask_l   19 non-null     float64                
 13  ask_c   19 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), 

In [36]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-21 13:00:00+00:00,21530,1.02646,1.02751,1.01536,1.02124,1.02638,1.02743,1.01527,1.02116,1.02655,1.02759,1.01544,1.02133
1,2022-07-21 14:00:00+00:00,14475,1.02123,1.02142,1.01684,1.01916,1.02115,1.02134,1.01675,1.01909,1.02131,1.02151,1.01694,1.01924
2,2022-07-21 15:00:00+00:00,7808,1.01916,1.02124,1.01824,1.02004,1.01907,1.02116,1.01816,1.01997,1.01925,1.02133,1.01833,1.02011
3,2022-07-21 16:00:00+00:00,5522,1.02003,1.02003,1.01750,1.01814,1.01996,1.01996,1.01743,1.01807,1.02010,1.02011,1.01757,1.01822
4,2022-07-21 17:00:00+00:00,4384,1.01818,1.01888,1.01756,1.01844,1.01810,1.01879,1.01736,1.01836,1.01825,1.01896,1.01774,1.01852
5,2022-07-21 18:00:00+00:00,3676,1.01844,1.02000,1.01832,1.01930,1.01837,1.01993,1.01824,1.01922,1.01851,1.02008,1.01841,1.01937
6,2022-07-21 19:00:00+00:00,3754,1.01932,1.02120,1.01916,1.02116,1.01924,1.02112,1.01909,1.02108,1.01939,1.02129,1.01924,1.02124
7,2022-07-21 20:00:00+00:00,2641,1.02114,1.02311,1.02050,1.02304,1.02106,1.02294,1.02041,1.02290,1.02122,1.02328,1.02058,1.02317
8,2022-07-21 21:00:00+00:00,376,1.02288,1.02293,1.02256,1.02280,1.02255,1.02275,1.02239,1.02264,1.02322,1.02343,1.02268,1.02297
9,2022-07-21 22:00:00+00:00,1764,1.02283,1.02284,1.02224,1.02240,1.02264,1.02275,1.02214,1.02230,1.02302,1.02302,1.02233,1.02249


In [37]:
# testing the code once again
status, data = fetch_candles(OANDA_URL, "GBP_AUD", count=10, granularity="H4")
candles_df = get_candles_df(data)
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-20 17:00:00+00:00,28043,1.73928,1.74026,1.73748,1.73902,1.73911,1.74008,1.73731,1.73881,1.73944,1.74044,1.73764,1.73922
1,2022-07-20 21:00:00+00:00,13224,1.73897,1.74004,1.73625,1.73722,1.73822,1.73984,1.73608,1.73701,1.73972,1.74066,1.73642,1.73744
2,2022-07-21 01:00:00+00:00,29966,1.73730,1.73985,1.73532,1.73542,1.73708,1.73966,1.73512,1.73522,1.73751,1.74004,1.73550,1.73561
3,2022-07-21 05:00:00+00:00,48473,1.73540,1.73962,1.73504,1.73843,1.73521,1.73947,1.73485,1.73827,1.73558,1.73980,1.73522,1.73859
4,2022-07-21 09:00:00+00:00,55233,1.73842,1.74123,1.73424,1.73616,1.73826,1.74104,1.73407,1.73602,1.73857,1.74142,1.73441,1.73631
5,2022-07-21 13:00:00+00:00,72653,1.73611,1.73648,1.72917,1.73302,1.73595,1.73631,1.72897,1.73284,1.73627,1.73666,1.72935,1.73320
6,2022-07-21 17:00:00+00:00,27248,1.73308,1.73491,1.72951,1.73066,1.73290,1.73474,1.72876,1.73040,1.73327,1.73508,1.72982,1.73091
7,2022-07-21 21:00:00+00:00,15263,1.73060,1.73308,1.72881,1.73240,1.72879,1.73290,1.72806,1.73222,1.73240,1.73329,1.72956,1.73257
8,2022-07-22 01:00:00+00:00,29464,1.73238,1.73350,1.72986,1.73326,1.73219,1.73332,1.72967,1.73307,1.73256,1.73369,1.73006,1.73345


In [38]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    9 non-null      datetime64[ns, tzutc()]
 1   volume  9 non-null      int64                  
 2   mid_o   9 non-null      float64                
 3   mid_h   9 non-null      float64                
 4   mid_l   9 non-null      float64                
 5   mid_c   9 non-null      float64                
 6   bid_o   9 non-null      float64                
 7   bid_h   9 non-null      float64                
 8   bid_l   9 non-null      float64                
 9   bid_c   9 non-null      float64                
 10  ask_o   9 non-null      float64                
 11  ask_h   9 non-null      float64                
 12  ask_l   9 non-null      float64                
 13  ask_c   9 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in

## Saving the fetched Candles in a Data File

In [41]:
def create_data_file(api_url, pair_name, count="10", granularity="H1"):
    
    status, data = fetch_candles(
        api_url, 
        pair_name, 
        count=count, 
        granularity=granularity
    )
    
    if status != 200:
        print("Failed", pair_name, data)
        return    
    
    if len(data) == 0:
        print("No candles", pair_name)
        return
    
    candles_df = get_candles_df(data)
    # safe compressed data frame to file
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [42]:
create_data_file(OANDA_URL, "USD_PLN", count=10, granularity="H4")

USD_PLN H4 9 candles, 2022-07-20 17:00:00+00:00 2022-07-22 01:00:00+00:00
